<a href="https://colab.research.google.com/github/Kunalchaudhary1/academicpages.github.io/blob/master/crop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d noulam/tomato

100% 742M/743M [00:26<00:00, 40.2MB/s]
100% 743M/743M [00:26<00:00, 29.6MB/s]


In [ ]:
import zipfile
zip_ref= zipfile.ZipFile('/content/tomato.zip','r')
zip_ref.extractall('/content')
zip_ref.close

<bound method ZipFile.close of <zipfile.ZipFile filename='/content/tomato.zip' mode='r'>>

In [ ]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from tensorflow import compat
from tensorflow import keras
from keras import Sequential
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,Activation
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
EPOCHS = 35
INIT_LR = 1e-3
BS = 35
default_image_size = tuple((100,100))
image_size = 0
directory_root = '/content/New Plant Diseases Dataset(Augmented)'
width=100
height=100
depth=3

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")

        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")

            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:250]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Processing Tomato___Early_blight ...
[INFO] Processing Tomato___Spider_mites Two-spotted_spider_mite ...
[INFO] Processing Tomato___Tomato_mosaic_virus ...
[INFO] Processing Tomato___healthy ...
[INFO] Processing Tomato___Septoria_leaf_spot ...
[INFO] Processing Tomato___Tomato_Yellow_Leaf_Curl_Virus ...
[INFO] Processing Tomato___Target_Spot ...
[INFO] Processing Tomato___Leaf_Mold ...
[INFO] Processing Tomato___Late_blight ...
[INFO] Processing Tomato___Bacterial_spot ...
[INFO] Processing Tomato___Early_blight ...
[INFO] Processing Tomato___Spider_mites Two-spotted_spider_mite ...
[INFO] Processing Tomato___Tomato_mosaic_virus ...
[INFO] Processing Tomato___healthy ...
[INFO] Processing Tomato___Septoria_leaf_spot ...
[INFO] Processing Tomato___Tomato_Yellow_Leaf_Curl_Virus ...
[INFO] Processing Tomato___Target_Spot ...
[INFO] Processing Tomato___Leaf_Mold ...
[INFO] Processing Tomato___Late_blight ...
[INFO] Processing Tomato___Bacterial_spot ...
[I

get size of processed image

In [ ]:
image_size = len(image_list)


In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)


In [ ]:
print(label_binarizer.classes_)


['Tomato___Bacterial_spot' 'Tomato___Early_blight' 'Tomato___Late_blight'
 'Tomato___Leaf_Mold' 'Tomato___Septoria_leaf_spot'
 'Tomato___Spider_mites Two-spotted_spider_mite' 'Tomato___Target_Spot'
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus' 'Tomato___Tomato_mosaic_virus'
 'Tomato___healthy']


In [ ]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0



In [ ]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)


[INFO] Spliting data to train, test


In [ ]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2,
    zoom_range=0.2,horizontal_flip=True,
    fill_mode="nearest")


In [ ]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.1))
model.add(Conv2D(64, (3, 3), padding="valid"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="valid"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="valid"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Conv2D(128, (3, 3), padding="valid"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.10))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.20))
model.add(Dense(n_classes))
model.add(Activation("sigmoid"))



In [ ]:
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 100, 100, 32)      896       
                                                                 
 activation_16 (Activation)  (None, 100, 100, 32)      0         
                                                                 
 batch_normalization_14 (Ba  (None, 100, 100, 32)      128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 33, 33, 32)        0         
 g2D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 33, 33, 32)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 31, 31, 64)       

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics= ['accuracy'])



In [ ]:
history = model.fit(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

Epoch 1/35
114/114 [==============================] - 19s 113ms/step - loss: 0.4761 - accuracy: 0.4560 - val_loss: 0.4002 - val_accuracy: 0.0960
Epoch 2/35
114/114 [==============================] - 16s 137ms/step - loss: 0.1903 - accuracy: 0.6459 - val_loss: 0.4301 - val_accuracy: 0.1870
Epoch 3/35
114/114 [==============================] - 14s 120ms/step - loss: 0.1566 - accuracy: 0.7072 - val_loss: 0.6310 - val_accuracy: 0.2010
Epoch 4/35
114/114 [==============================] - 13s 111ms/step - loss: 0.1259 - accuracy: 0.7745 - val_loss: 0.4126 - val_accuracy: 0.3020
Epoch 5/35
114/114 [==============================] - 13s 110ms/step - loss: 0.1185 - accuracy: 0.7851 - val_loss: 0.2300 - val_accuracy: 0.6190
Epoch 6/35
114/114 [==============================] - 13s 114ms/step - loss: 0.1049 - accuracy: 0.8179 - val_loss: 0.1755 - val_accuracy: 0.7090
Epoch 7/35
114/114 [==============================] - 13s 110ms/step - loss: 0.0980 - accuracy: 0.8303 - val_loss: 0.2654 - val_ac

In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")


[INFO] Calculating model accuracy
32/32 [==============================] - 0s 8ms/step - loss: 0.1434 - accuracy: 0.8250
Test Accuracy: 82.4999988079071


In [ ]:
model.save('my_mode.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
scores = model.evaluate(x_train, y_train)
print(f"Test Accuracy: {scores[1]*100}")


ValueError: ignored

In [ ]:
import keras
import cv2
import matplotlib.pyplot as plt


In [ ]:
import numpy as np
CATEGORIES = ['Tomato___Bacterial_spot','Tomato___Early_blight',
 'Tomato___Late_blight' ,'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite' ,'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus' ,'Tomato___healthy']

def image(path):
    img = cv2.imread(path)
    new_arr = cv2.resize(img,(100, 100))
    new_arr = np.array(new_arr/255)
    new_arr = new_arr.reshape(-1, 100, 100, 3)
    return new_arr

model = keras.models.load_model('/content/my_model.h5')


In [ ]:
prediction = model.predict(image('/content/th (1).jpeg'))

print(CATEGORIES[prediction.argmax()])


1/1 [==============================] - 0s 110ms/step
Tomato___Target_Spot
